In [1]:
import pyspark
import pyspark.sql.functions as F
import pandas as pd
import numpy as np
import re

from pydataset import data

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [3]:
p_df = pd.DataFrame(
    dict(language=['python', 'C#'])
)

# Create a spark data frame that contains your favorite programming languages.

In [4]:
sp_df = spark.createDataFrame(p_df)

In [5]:
sp_df

DataFrame[language: string]

In [6]:
sp_df.show(2)

+--------+
|language|
+--------+
|  python|
|      C#|
+--------+



In [7]:
print((sp_df.count(), len(sp_df.columns)))

(2, 1)


In [8]:
sp_df.printSchema()

root
 |-- language: string (nullable = true)



In [9]:
sp_df.show(5)

+--------+
|language|
+--------+
|  python|
|      C#|
+--------+



In [10]:
mpg = data('mpg')

In [11]:
mpg_sp = spark.createDataFrame(mpg)

In [12]:
mpg_sp.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [13]:
from pyspark.sql.functions import lit, concat
mpg_sp.select(concat(lit('The '), mpg_sp.year, lit(' '), mpg_sp.manufacturer, lit(' '), 
                     mpg_sp.model, lit(' has a '), mpg_sp.cyl, lit(' cylinder engine'))
              .alias("output")).show(5, False)

+----------------------------------------+
|output                                  |
+----------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine|
|The 1999 audi a4 has a 4 cylinder engine|
|The 2008 audi a4 has a 4 cylinder engine|
|The 2008 audi a4 has a 4 cylinder engine|
|The 1999 audi a4 has a 6 cylinder engine|
+----------------------------------------+
only showing top 5 rows



In [26]:
from pyspark.sql.functions import regexp_extract, regexp_replace, when

In [31]:
mpg_sp.select(
    regexp_extract('trans', r'^(\w+)\(', 1).alias('trans'),
    when(mpg_sp.trans.like('auto%'), 'auto').otherwise('manual')
).show()

+------+----------------------------------------------------+
| trans|CASE WHEN trans LIKE auto% THEN auto ELSE manual END|
+------+----------------------------------------------------+
|  auto|                                                auto|
|manual|                                              manual|
|manual|                                              manual|
|  auto|                                                auto|
|  auto|                                                auto|
|manual|                                              manual|
|  auto|                                                auto|
|manual|                                              manual|
|  auto|                                                auto|
|manual|                                              manual|
|  auto|                                                auto|
|  auto|                                                auto|
|manual|                                              manual|
|  auto|

The name of the column should be language

View the schema of the dataframe

Output the shape of the dataframe

Show the first 5 records in the dataframe

Load the mpg dataset as a spark dataframe.

Create 1 column of output that contains a message like the one below:
The 1999 audi a4 has a 4 cylinder engine.
For each vehicle.

Transform the trans column so that it only contains either manual or auto.

Load the tips dataset as a spark dataframe.

In [32]:
tips = data('tips')

In [33]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


In [35]:
tips = spark.createDataFrame(tips)

TypeError: data is already a DataFrame

In [36]:
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



What percentage of observations are smokers?

In [42]:
sum_spend = tips.agg(F.sum(F.col("smoker")).alias("smokes")).collect()[0][0]

+----------+----+------+------+----+------+----+
|total_bill| tip|   sex|smoker| day|  time|size|
+----------+----+------+------+----+------+----+
|     16.99|1.01|Female|  null| Sun|Dinner|   2|
|     10.34|1.66|  Male|  null| Sun|Dinner|   3|
|     21.01| 3.5|  Male|  null| Sun|Dinner|   3|
|     23.68|3.31|  Male|  null| Sun|Dinner|   2|
|     24.59|3.61|Female|  null| Sun|Dinner|   4|
|     25.29|4.71|  Male|  null| Sun|Dinner|   4|
|      8.77| 2.0|  Male|  null| Sun|Dinner|   2|
|     26.88|3.12|  Male|  null| Sun|Dinner|   4|
|     15.04|1.96|  Male|  null| Sun|Dinner|   2|
|     14.78|3.23|  Male|  null| Sun|Dinner|   2|
|     10.27|1.71|  Male|  null| Sun|Dinner|   2|
|     35.26| 5.0|Female|  null| Sun|Dinner|   4|
|     15.42|1.57|  Male|  null| Sun|Dinner|   2|
|     18.43| 3.0|  Male|  null| Sun|Dinner|   4|
|     14.83|3.02|Female|  null| Sun|Dinner|   2|
|     21.58|3.92|  Male|  null| Sun|Dinner|   2|
|     10.33|1.67|Female|  null| Sun|Dinner|   3|
|     16.29|3.71|  M

Create a column that contains the tip percentage
Calculate the average tip percentage for each combination of sex and smoker.

Use the seattle weather dataset referenced in the lesson to answer the questions below.

Convert the temperatures to farenheight.
Which month has the most rain, on average?
Which year was the windiest?
What is the most frequent type of weather in January?
What is the average high and low tempurature on sunny days in July in 2013 and 2014?
What percentage of days were rainy in q3 of 2015?
For each year, find what percentage of days it rained (had non-zero precipitation).